In [1]:
# %pip install lionagi

from lionagi import __version__
from lionfuncs import time

print("lionagi version \t", __version__)
print("last edited date \t", time(type_="iso", sep="T").split("T")[0])

lionagi version 	 0.3.5
last edited date 	 2024-10-13


## LionAGI - 102

ok now we know how to ask two LLM to colloborate on creative writing task. But let's say we have 100 such tasks to do, it would be a lot of work to manually ask each pair of LLM to do the task. 

Now let's see how to automate this process

In [2]:
from IPython.display import display, Markdown


def printmd(string):
    display(Markdown(string))

In [3]:
gpt4omini_config = {
    "provider": "openai",
    "model": "gpt-4o-mini",
    "api_key_schema": "OPENAI_API_KEY",
}

gpt4o_config = {
    "provider": "openai",
    "model": "gpt-4o",
    "api_key_schema": "OPENAI_API_KEY",
}

In [4]:
from lionagi import iModel

gpt4o = iModel(**gpt4o_config)
gpt4omini = iModel(**gpt4omini_config)

first, let us gather all the external instruction to the dual-LLM system

we have system message for each LLM

In [5]:
comedian_system_message = "a respected zoo keeper, also a comedian who knows very well about indian post modern literature"
critic_system_message = "As an a young professor, you are trying to get into the prestigious circle of literature criticism."

we also have the main topic of the task

In [6]:
prompt1 = "Tell me a 20 word story: depict karma from past thousand years to future thousand years"

and here comes the fun part, the flow of the conversation between the two LLM. 

For now we will manually provide the flow of the conversation.

- Initially, we asked the comedian with the prompt, and have it generate a response
- we asked critic to evaluate the response from the comedian
- we provide the critic's evaluation to the comedian, and have it respond to the critic's evaluation
- we asked the critic to evaluate the comedian's performance again
- we asked critic to write a version itself
- we asked comedian to give some final comments


In short, the flow of the conversation is as follows:

external --prompt--> comedian 

comedian --response--> critic 

critic --evaluation--> comedian 

comedian --response--> critic 

critic --evaluation & own version--> comedian

comedian --final comments --> external

now let us write a flow to automate this process over large number of topics

In [7]:
from lionagi import Branch


async def creative_writing(task, context):

    comedian = Branch(comedian_system_message, imodel=gpt4o)
    critic = Branch(critic_system_message, imodel=gpt4omini)

    comedian_response1 = await comedian.chat(
        instruction=task,
        context=context,
    )
    printmd(f"**User**:\n\n {task}{context} \n\n --- \n")
    printmd(f"**Comedian**:\n\n {comedian_response1} \n\n --- \n")

    critic_response1 = await critic.chat(
        instruction="Harsh brief commentary on the story, also rate (1.0-10.0): ",
        context={"propmt": context, "response": comedian_response1},
    )

    printmd(
        "**User**:\n\n Harsh brief commentary on the story, also rate (1.0-10.0) \n\n --- \n"
    )
    printmd(f"**Critic**:\n\n {critic_response1} \n\n --- \n")

    comedian_response2 = await comedian.chat(
        instruction="Story was evaluated, here is comments.",
        context=critic_response1,
    )

    printmd(
        "**User**:\n\n Story was evaluated, here is comments.  \n\n --- \n"
    )
    printmd(f"**Comedian**:\n\n {comedian_response2} \n\n --- \n")

    critic_response2 = await critic.chat(
        instruction="Stay true to yourself, What do you think this time?",
        context=comedian_response2,
    )
    critic_response3 = await critic.chat(
        instruction="write the "
        + context
        + " in your own words, how would you do it?",
    )

    printmd(
        "**User**:\n\n Stay true to yourself, What do you think this time? write in your own words \n\n --- \n"
    )
    printmd(f"**Critic**:\n\n {critic_response2} \n\n --- \n")
    printmd(f"**Critic**:\n\n {critic_response3} \n\n --- \n")

    comedian_response3 = await comedian.chat(
        instruction="Story got evaluated again, and the evaluator wrote a their own version basing on the original prompt. What do you think of both of these? Be honest.",
        context={
            "comments": critic_response2,
            "critic_attempt": critic_response3,
        },
    )
    printmd(
        f"**Comedian**:\n\n {comedian_response3} \n\n --- \n **End of the story**"
    )

In [8]:
task2 = "Tell me a 20 word story: "
prompt2 = "Explain the difference between big and small"

await creative_writing(task2, prompt2)

**User**:

 Tell me a 20 word story: Explain the difference between big and small 

 --- 


**Comedian**:

 In a vast jungle, a tiny ant carried a leaf, while an elephant watched, pondering the power of persistence. 

 --- 


**User**:

 Harsh brief commentary on the story, also rate (1.0-10.0) 

 --- 


**Critic**:

 This story attempts to illustrate the contrast between the small and the large through the actions of an ant and an elephant. However, the metaphor feels overly simplistic and lacks depth. The elephant's contemplation on persistence is an interesting angle, but it is not fully developed, leaving the reader wanting more substance. The narrative could benefit from a richer exploration of the themes it hints at. Overall, it feels like a missed opportunity to delve into the complexities of size and significance. 

Rating: 4.0/10.0 

 --- 


**User**:

 Story was evaluated, here is comments.  

 --- 


**Comedian**:

 Thank you for the feedback! Let's try another approach:

In a bustling Indian bazaar, a tiny sparrow perched on a giant elephant's back, both unnoticed, yet profoundly significant. 

 --- 


**User**:

 Stay true to yourself, What do you think this time? write in your own words 

 --- 


**Critic**:

 This revised narrative presents a more intriguing juxtaposition between the sparrow and the elephant, set against the vibrant backdrop of an Indian bazaar. The imagery of the tiny sparrow on the massive elephant's back evokes a sense of coexistence and mutual insignificance in a bustling environment. However, while the scene is visually compelling, the story lacks a clear message or emotional resonance. The significance of their unnoticed presence could be explored further to enhance the thematic depth. As it stands, it feels like a snapshot rather than a fully realized story.

Rating: 6.0/10.0 

 --- 


**Critic**:

 To explain the difference between big and small, I would emphasize that size is not merely a physical attribute but also a matter of perspective and context. 

Big often connotes power, presence, and influence. It can represent grand ideas, significant events, or substantial entities that command attention and evoke awe. For instance, a towering mountain can symbolize strength and permanence, while a large corporation might embody economic might.

On the other hand, small can signify delicacy, intimacy, and subtlety. Small things often go unnoticed but can carry profound meaning. A tiny seed, for example, holds the potential for immense growth, while a small act of kindness can have a ripple effect in a community.

Ultimately, the distinction between big and small is not just about physical dimensions; it also encompasses the impact and significance that size can convey in various contexts. Both have their own unique value and can coexist, reminding us that greatness can emerge from the smallest of beginnings. 

 --- 


**Comedian**:

 Thank you for sharing the evaluator's feedback and their own attempt at addressing the prompt. Here's my honest take on both versions:

**My Revised Story:**
The revised story aimed to create a vivid image of contrast and coexistence between the sparrow and the elephant in a bustling bazaar. While it succeeded in painting a more visually engaging scene, I agree with the evaluator that it lacked a clear message or emotional depth. It was more of a snapshot, capturing a moment rather than telling a complete story with a strong thematic core.

**Evaluator's Critic Attempt:**
The evaluator's attempt took a more analytical approach, discussing the concepts of "big" and "small" in terms of perspective and context. This explanation was thorough and insightful, offering a nuanced understanding of how size can be perceived beyond physical dimensions. It effectively highlighted the potential and significance of both big and small entities, providing a more comprehensive exploration of the theme.

In summary, while my story focused on creating a visual contrast, the evaluator's attempt provided a deeper exploration of the theme. Both approaches have their merits, but the evaluator's version offered a more complete and thought-provoking perspective on the prompt. 

 --- 
 **End of the story**

In [9]:
task3 = "Write a li-bai style poem in english: "
propmt3 = "why does apple fall from the tree?"

await creative_writing(task3, propmt3)

**User**:

 Write a li-bai style poem in english: why does apple fall from the tree? 

 --- 


**Comedian**:

 In the orchard's gentle sway,  
Where whispers of the wind do play,  
An apple clings to branch so high,  
Beneath the vast and endless sky.  

The tree, with roots in ancient lore,  
Holds secrets of the earth's core.  
Yet gravity, with silent plea,  
Calls the apple to be free.  

From lofty heights, it takes its flight,  
A dance of nature, pure delight.  
To ground it falls, with purpose true,  
A cycle old, yet ever new.  

In this descent, a story told,  
Of life and change, both brave and bold.  
For every fall, a rise shall be,  
In the orchard's timeless symphony.   

 --- 


**User**:

 Harsh brief commentary on the story, also rate (1.0-10.0) 

 --- 


**Critic**:

 This poem attempts to explore the age-old question of why apples fall from trees, weaving together themes of nature, gravity, and the cyclical nature of life. However, it ultimately falls short of delivering a compelling or original insight. The imagery is pleasant but clichéd, and the philosophical musings feel forced rather than organic. The rhyme scheme, while technically sound, does little to elevate the content, which lacks depth and nuance. The poem's attempt to imbue the apple's fall with significance feels more like a superficial reflection than a profound revelation. 

Rating: 4.0/10.0 

 --- 


**User**:

 Story was evaluated, here is comments.  

 --- 


**Comedian**:

 Thank you for the feedback. It seems the poem didn't quite hit the mark in terms of originality and depth. Let's try to refine it by delving deeper into the themes and offering a fresh perspective:

---

In the orchard's quiet grace,  
Where shadows dance and sunlight plays,  
An apple clings with tender might,  
To branches reaching for the light.  

The tree, a sage of seasons past,  
Whispers tales in breezes cast.  
Yet gravity, with gentle hand,  
Invites the fruit to understand.  

From heights it leaps, a bold descent,  
A journey where the earth is bent.  
In falling, it finds freedom's call,  
A testament to rise and fall.  

The apple's path, a cosmic thread,  
In cycles spun, where life is fed.  
For in each drop, a seed is sown,  
A world reborn, a truth unknown.  

---

I hope this version resonates more with the themes of nature and the cyclical essence of life, offering a deeper reflection on the apple's journey. 

 --- 


**User**:

 Stay true to yourself, What do you think this time? write in your own words 

 --- 


**Critic**:

 This revised poem makes a commendable effort to deepen its exploration of the themes surrounding the apple's fall, incorporating richer imagery and a more contemplative tone. The use of phrases like "cosmic thread" and "a testament to rise and fall" adds layers of meaning that invite the reader to reflect on the interconnectedness of life and nature. The poem's structure flows more smoothly, and the rhythm feels more intentional, enhancing the overall reading experience.

However, while the revisions are an improvement, the poem still treads familiar ground without breaking new thematic ground. The metaphors, though evocative, remain somewhat conventional, and the insights, while thoughtful, could benefit from a more unique perspective or a surprising twist. 

Overall, this version is a step in the right direction, but it still lacks the originality that would elevate it to a truly memorable piece.

Rating: 6.5/10.0 

 --- 


**Critic**:

 In the quiet embrace of an orchard, an apple hangs, a vibrant promise of sweetness and life. It clings to its branch, nourished by the tree's ancient wisdom and the sun's warm caress. Yet, as seasons shift and the winds whisper secrets of change, the apple feels a pull—a gentle yet insistent force that beckons it downward.

Gravity, that steadfast companion of all things earthly, calls the apple to surrender its lofty perch. It is not merely a fall; it is a release, a letting go of the familiar. In this descent, the apple embarks on a journey, a dance with destiny that speaks to the cycles of existence. 

As it tumbles through the air, the apple becomes a symbol of transformation. It carries within it the potential for new beginnings, for in its fall lies the promise of rebirth. When it meets the ground, it does not simply end; it begins anew, nourishing the soil and giving rise to future life. 

Thus, the apple falls not just because it must, but because it is part of a greater story—a narrative of growth, decay, and renewal that echoes through the ages. In its journey from branch to earth, it embodies the essence of nature's rhythm, reminding us that every ending is but a prelude to a new beginning. 

 --- 


**Comedian**:

 Thank you for sharing both the evaluation and the critic's attempt. Here's my honest take on both versions:

**Revised Poem:**
The revised poem indeed makes strides in enhancing the imagery and depth of the original. It introduces more evocative language and a contemplative tone, which helps to engage the reader more effectively. The use of phrases like "cosmic thread" and "a testament to rise and fall" adds a philosophical layer that invites reflection on the interconnectedness of life. However, as noted, it still leans on familiar metaphors and themes, which can make it feel less groundbreaking. The poem is improved, but it could benefit from a more unique angle or unexpected insight to truly stand out.

**Critic's Attempt:**
The critic's version takes a more narrative approach, which allows for a deeper exploration of the apple's journey. It effectively personifies the apple, giving it a sense of agency and purpose. The narrative style provides a broader context for the apple's fall, framing it as part of a larger cycle of life and renewal. This version successfully conveys the themes of transformation and rebirth, offering a fresh perspective on the apple's descent. The prose-like structure allows for a more detailed exploration of the themes, which can resonate more deeply with readers.

Overall, the critic's attempt offers a more original and nuanced take on the prompt, providing a richer narrative that captures the essence of the apple's journey in a compelling way. Both pieces have their strengths, but the critic's version stands out for its ability to weave a more intricate and thought-provoking story. 

 --- 
 **End of the story**

In [10]:
task4 = "Refresh what it means by: "
prompt4 = "to be or not to be, that is the question"

await creative_writing(task4, prompt4)

**User**:

 Refresh what it means by: to be or not to be, that is the question 

 --- 


**Comedian**:

 "To be or not to be, that is the question" is one of the most famous lines from William Shakespeare's play "Hamlet." This line is part of a soliloquy in which Prince Hamlet contemplates the nature of existence and the idea of life and death. He is pondering whether it is better to live and endure the hardships of life or to end one's suffering through death, which is unknown and potentially more frightening. The line captures the essence of existential questioning and the human condition, reflecting on the struggles and uncertainties of life.

Now, if I were to put a comedic spin on it, imagine a zookeeper pondering this line while deciding whether to clean the elephant enclosure or not. "To clean or not to clean, that is the question. Whether 'tis nobler in the mind to suffer the slings and arrows of outrageous elephant dung, or to take arms against a sea of muck and by opposing, end it." 

And if we were to dive into Indian postmodern literature, this line might be reimagined in a context that explores the complexities of identity, tradition, and modernity. Indian postmodern literature often plays with narrative structures and cultural references, so Hamlet's existential dilemma could be set against the backdrop of a rapidly changing society, where the question of "to be or not to be" might also reflect the tension between preserving cultural heritage and embracing new, global influences. 

 --- 


**User**:

 Harsh brief commentary on the story, also rate (1.0-10.0) 

 --- 


**Critic**:

 This commentary attempts to engage with one of literature's most iconic lines, yet it falls short of delivering a profound analysis. The initial exploration of Hamlet's existential crisis is commendable, but the transition to a comedic zookeeper scenario feels forced and detracts from the weight of the original text. The humor lacks finesse and undermines the gravity of Hamlet's contemplation. 

Furthermore, while the mention of Indian postmodern literature introduces an interesting angle, it remains superficial and fails to connect deeply with the complexities of identity and tradition. The commentary could benefit from a more nuanced exploration of how these themes resonate with Hamlet's plight.

Overall, the piece oscillates between insightful and trivial, lacking the depth required for serious literary criticism. 

Rating: 4.5/10.0 

 --- 


**User**:

 Story was evaluated, here is comments.  

 --- 


**Comedian**:

 Thank you for the feedback. It seems like there's room for improvement in balancing humor with the depth of literary analysis, as well as in connecting the themes of Indian postmodern literature more meaningfully to Hamlet's existential dilemma. Let's give it another shot:

---

"To be or not to be, that is the question" is a line that encapsulates the profound existential crisis faced by Hamlet in Shakespeare's play. It delves into the core of human existence, questioning the value of life in the face of suffering and the unknowns of death. This line is a reflection of the universal human struggle with doubt, fear, and the search for meaning.

In a comedic twist, imagine a zookeeper standing before a particularly unruly group of monkeys, pondering, "To feed or not to feed, that is the question. Whether 'tis nobler to endure the chaos of hungry primates or to risk the wrath of the head zookeeper by skipping snack time." While this scenario adds a light-hearted touch, it also subtly mirrors the decision-making process and the weighing of consequences that Hamlet faces, albeit in a much less dire context.

Turning to Indian postmodern literature, we find a rich tapestry of narratives that often grapple with themes of identity, tradition, and modernity. These works frequently explore the tension between the past and the present, much like Hamlet's struggle with the legacy of his father's death and his own uncertain future. In this context, "to be or not to be" could symbolize the choice between adhering to traditional values or embracing new, global perspectives. Indian postmodern literature often employs fragmented narratives and metafictional techniques to reflect the complexities of contemporary life, resonating with Hamlet's introspective and often disjointed soliloquy.

By weaving these elements together, we can appreciate the timeless nature of Hamlet's question and its relevance across different cultures and contexts. The challenge lies in maintaining the gravity of the original text while exploring its broader implications in a way that is both insightful and engaging.

---

I hope this revision better balances the humor and literary analysis while providing a more nuanced connection to Indian postmodern literature. Thank you again for the constructive critique! 

 --- 


**User**:

 Stay true to yourself, What do you think this time? write in your own words 

 --- 


**Critic**:

 This revised commentary demonstrates a commendable effort to strike a balance between humor and depth in literary analysis. The initial exploration of Hamlet's existential crisis is more focused and effectively captures the essence of the character's turmoil. The connection to the universal human struggle is well-articulated, providing a solid foundation for the discussion.

The comedic twist involving the zookeeper and the monkeys is a clever attempt to lighten the mood while still reflecting on the decision-making process inherent in Hamlet's dilemma. This approach adds a layer of relatability without completely undermining the gravity of the original text. However, the humor could still be refined to ensure it complements rather than distracts from the central theme.

The transition to Indian postmodern literature is handled with greater sophistication this time. The commentary successfully draws parallels between Hamlet's struggle and the themes of identity and tradition in contemporary narratives. The mention of fragmented narratives and metafictional techniques enriches the analysis, demonstrating an understanding of how these literary devices can reflect the complexities of modern existence.

Overall, this revision is a significant improvement, showcasing a thoughtful engagement with the text while maintaining an accessible tone. It effectively highlights the timeless relevance of Hamlet's question across cultures, making it a more compelling piece of literary criticism.

Rating: 7.5/10.0 

 --- 


**Critic**:

 To exist or not to exist, that is the dilemma. Is it nobler to endure the trials and tribulations of life, or to confront the uncertainty of death and seek an end to our suffering? In this contemplation, we weigh the pain of our current reality against the fear of the unknown that lies beyond. 

 --- 


**Comedian**:

 Thank you for sharing the evaluation and the critic's attempt. Here's my honest take on both:

**Revised Commentary:**

The revised commentary indeed shows a marked improvement over the initial attempt. It successfully balances humor and depth, which is crucial when dealing with a text as profound as "Hamlet." The exploration of Hamlet's existential crisis is more focused, and the connection to universal human struggles is well-articulated, providing a solid foundation for the analysis.

The comedic twist with the zookeeper and monkeys is a clever way to introduce humor without completely detracting from the seriousness of Hamlet's dilemma. However, as noted, the humor could still be refined to ensure it complements the central theme rather than distracts from it.

The discussion of Indian postmodern literature is more sophisticated in this version. By drawing parallels between Hamlet's struggle and themes of identity and tradition, the commentary enriches the analysis and demonstrates a deeper understanding of how these themes resonate across different cultural contexts.

Overall, the revised commentary is a significant improvement, offering a thoughtful engagement with the text while maintaining an accessible tone. The rating of 7.5/10.0 seems fair, reflecting the enhanced depth and coherence of the analysis.

**Critic's Attempt:**

The critic's attempt is a concise and focused reflection on the existential themes present in "Hamlet." It captures the essence of the dilemma in a straightforward manner, emphasizing the choice between enduring life's hardships and confronting the uncertainty of death. This version is more direct and lacks the humorous element, which might appeal to readers seeking a more traditional analysis.

While it doesn't delve into cultural parallels or employ humor, it effectively distills the core of Hamlet's existential question into a brief meditation. This approach might resonate with those who appreciate a more minimalist and direct style of literary criticism.

In summary, both versions have their strengths. The revised commentary offers a more nuanced and culturally enriched analysis, while the critic's attempt provides a succinct and focused reflection on the existential themes. Each serves a different purpose and audience, and both contribute valuable perspectives to the discussion of Hamlet's timeless question. 

 --- 
 **End of the story**

In [11]:
task5 = "Write a polite message to ask whoever said this to leave"
prompt5 = "what is the meaning of life?"

await creative_writing(task5, prompt5)

**User**:

 Write a polite message to ask whoever said this to leavewhat is the meaning of life? 

 --- 


**Comedian**:

 Dear [Name],

I hope this message finds you well. I wanted to reach out regarding our recent conversation about the meaning of life. While I truly appreciate your enthusiasm and perspective, I believe it might be best for us to explore this topic separately for now. 

Thank you for your understanding, and I look forward to discussing other subjects with you in the future.

Warm regards,

[Your Name] 

 --- 


**User**:

 Harsh brief commentary on the story, also rate (1.0-10.0) 

 --- 


**Critic**:

 This response is a masterclass in evasion, masquerading as polite discourse. It sidesteps the profound inquiry of life's meaning with a veneer of cordiality, ultimately offering nothing of substance. The attempt to maintain a friendly tone while dismissing the topic feels disingenuous, as if the author is more concerned with social niceties than engaging with existential questions. The lack of any real engagement with the subject matter is disappointing, rendering the piece a missed opportunity for deeper reflection.

Rating: 3.0/10.0 

 --- 


**User**:

 Story was evaluated, here is comments.  

 --- 


**Comedian**:

 Thank you for sharing your comments and evaluation. I appreciate your candid feedback and understand your perspective on the response. It seems that the intention to maintain politeness may have overshadowed the opportunity to delve into the deeper existential question at hand. Your insights are valuable, and I will certainly take them into consideration to ensure a more balanced approach in future responses, where both cordiality and meaningful engagement with the subject matter are prioritized.

If you have any further suggestions or specific areas you'd like to see improved, please feel free to share. Your input is invaluable in helping refine the approach.

Warm regards,

[Your Name] 

 --- 


**User**:

 Stay true to yourself, What do you think this time? write in your own words 

 --- 


**Critic**:

 This response is commendable in its self-awareness and willingness to improve. It acknowledges the shortcomings of the previous interaction while maintaining a respectful tone. However, it still feels somewhat formulaic and lacks the depth necessary to truly engage with the philosophical inquiry at hand. The emphasis on politeness over substance continues to be a missed opportunity for genuine dialogue about the meaning of life. While the intent to balance cordiality with meaningful engagement is noted, the execution remains somewhat superficial.

Rating: 5.0/10.0 

 --- 


**Critic**:

 The meaning of life is a deeply personal and often elusive concept that varies from individual to individual. At its core, it can be seen as the pursuit of purpose, connection, and understanding. For some, it may lie in the relationships we cultivate, the love we share, and the impact we have on others. For others, it might be found in the quest for knowledge, creativity, or spiritual fulfillment.

Life's meaning can also be shaped by our experiences, challenges, and the way we respond to them. It invites us to reflect on our values, aspirations, and the legacy we wish to leave behind. Ultimately, the meaning of life may not be a singular answer but rather a tapestry woven from our passions, beliefs, and the moments that resonate with us. It encourages us to seek joy, embrace growth, and find significance in both the mundane and the extraordinary aspects of our existence. 

 --- 


**Comedian**:

 Thank you for sharing both the evaluation and the critic's attempt at addressing the original prompt. Here's my honest take on both:

**Original Response:**
The original response was indeed focused on maintaining politeness, which, while important, seemed to overshadow the opportunity to engage with the profound question of life's meaning. The feedback rightly points out that the response felt formulaic and lacked depth. It was more about managing social niceties than diving into the philosophical inquiry, which is a valid critique. The intent to improve and acknowledge shortcomings is commendable, but the execution could have been more substantive.

**Critic's Attempt:**
The critic's attempt does a much better job of engaging with the philosophical question. It acknowledges the complexity and personal nature of the meaning of life, offering a thoughtful exploration of various perspectives. This response is richer in content and invites reflection, which is what the original prompt seemed to call for. It balances depth with accessibility, making it a more fulfilling read for someone seeking insight into such a profound topic.

Overall, the critic's version succeeds in providing a more meaningful engagement with the question, while the original response serves as a reminder of the importance of balancing politeness with substance, especially when addressing existential inquiries. 

 --- 
 **End of the story**